In [ ]:
from utils import *
data_ID = "60a6_He_Be8"
num_orbitals = (8,8) # Consider helium core, deduct 
num_neut_orb = num_orbitals[0]

### The following doesnt require setting up, unless file naming is different
obs_onebody_csv = data_ID+"/"+data_ID+"-1B_H_input.csv"
obs_twobody_csv = data_ID+"/"+data_ID+"-2B_H_input.csv"
parameter_py = data_ID+"/"+"parameter.txt"

# Onebody terms

In [ ]:
source = "source\Be8_600\Vpair_800MeV_Be8_include_all_2b_transitions_one.txt"
text_one = []
with open(source, 'rt') as myfile:    #filename
# with open("source/Be8_SIII_new/Vpair_Vdelta_800.txt", 'rt') as myfile:    #filename
    for line in myfile:
        line = line.rstrip('\n')
        text_one.append(line)
text_one = text_one[1:]

In [ ]:
obs_onebody_df = pd.DataFrame(columns =['q_i','q_f','epsilon'])
prot_start = 6
for counter, i in enumerate(text_one):
    split = i.split()
    fina = int(split[0])-1 if split[1]=="1" else int(split[0])-1+num_neut_orb
    init = int(split[3])-1 if split[4]=="1" else int(split[3])-1+num_neut_orb
    if fina in [0,1,8,9]: continue
        
    epsilon = round(float(split[-1]) * float(split[-2]) , 5 )
    if fina != init :
        continue
    elif 2 <= init < 8:
        obs_onebody_df.loc[counter] = [init-2, fina-2, epsilon]
    elif 8 <= init:
        obs_onebody_df.loc[counter] = [init-4, fina-4, epsilon]


In [ ]:
obs_onebody_df=obs_onebody_df.sort_values(["q_i"])
obs_onebody_df = obs_onebody_df.reset_index()
obs_onebody_df = obs_onebody_df[["q_i","q_f","epsilon"]]
obs_onebody_df

# Twobody terms

In [ ]:
text_two = []
with open("source\Be8_600\Vpair_800MeV_Be8_include_all_2b_transitions_two.txt", 'rt') as myfile:    #filename
# with open("source/Be8_SIII_new/Vpair_Vdelta_800.txt", 'rt') as myfile:    #filename
    for line in myfile:
        line = line.rstrip('\n')
        text_two.append(line)

In [ ]:
line_num = identify_line(text_two, 'V0 \(1\)', "V0 \(2\)")
text_two_neut = text_two[line_num[0]+2: line_num[1]]
text_two_prot = text_two[line_num[1]+1:]

In [ ]:
pairlist = list(set(degenerate_pair_gen(num_orbitals)) & set(sin_bod_if_list_gen(num_orbitals)))
pairlist_reversed = []
for i in pairlist:
    pairlist_reversed.append((i[1],i[0]))
pairlist += pairlist_reversed ; pairlist.sort()
pairlist

In [ ]:
# to process all possible values (60a6_He_Be8) Stetcu style ( cosider an inert helium core)
obs_twobody_df = pd.DataFrame(columns =["q_i1", "q_i2", "q_f1", "q_f2", "V_ffii"])
neut_start = 0
prot_start = 6
# taking the first two as init and the last 2 as final.
count = 0
for counter, i in enumerate(text_two_neut):
    split = i.split()
    fina = (int(split[0])-1 -2,int(split[2])-1 -2);
    init = (int(split[4])-1 -2,int(split[6])-1 -2)
    if ((abs(float(split[-1])) < 0.0001) 
        or fina[0]<neut_start or fina[1]<neut_start 
        or init[0]<neut_start or init[1]<neut_start):
        count +=1 
        pass
    else:
        V_pair = round(float(split[-1]) , 5 )
        obs_twobody_df.loc[counter] = [init[0],init[1],fina[0],fina[1],V_pair]

dum_len = len(obs_twobody_df)

for counter, i in enumerate(text_two_prot):
    split = i.split()
    fina = (int(split[0])-1+prot_start-2,int(split[2])-1+prot_start-2);
    init = (int(split[4])-1+prot_start-2,int(split[6])-1+prot_start-2) #  -1 because start from 0, plus prot_start because proton comes after neutron's
    if ((abs(float(split[-1])) <0.0001)
        or fina[0]<prot_start or fina[1]<prot_start 
        or init[0]<prot_start or init[1]<prot_start):
        count +=1 
        pass
    else:
        V_pair = round(float(split[-1]) , 5 )
        obs_twobody_df.loc[counter+dum_len] = [init[0],init[1],fina[0],fina[1],V_pair]


In [ ]:
pd.set_option('display.max_rows', None)

obs_twobody_df=obs_twobody_df.sort_values(["q_f1","q_i1"])
obs_twobody_df = obs_twobody_df.reset_index()
obs_twobody_df = obs_twobody_df[["q_i1","q_i2","q_f1","q_f2","V_ffii"]]
obs_twobody_df

In [ ]:
obs_onebody_df.to_csv(obs_onebody_csv)
obs_twobody_df.to_csv(obs_twobody_csv)

In [ ]:
# To process onebody for (60c1 and 60a1)
# obs_onebody_df = pd.DataFrame(columns =['q_i','q_f','epsilon'])

# for counter, i in enumerate(text_one):
#     split = i.split()
#     fina = int(split[0])-1 if split[1]=="1" else int(split[0])-1+num_neut_orb
#     init = int(split[3])-1 if split[4]=="1" else int(split[3])-1+num_neut_orb
#     epsilon = round(float(split[-1]) * float(split[-2]) , 5 )
# #     if epsilon in obs_onebody_df["epsilon"].tolist():
# #         continue
#     obs_onebody_df.loc[counter] = [init, fina, epsilon]

In [ ]:
# ## to process only negative values ( 60c1)
# obs_twobody_df = pd.DataFrame(columns =["q_i1", "q_i2", "q_f1", "q_f2", "V_ffii"])

# # taking the first two as init and the last 2 as final.

# for counter, i in enumerate(text_two_neut):
#     split = i.split()
#     fina = (int(split[0])-1,int(split[2])-1); 
#     init =  (int(split[4])-1,int(split[6])-1)
#     V_pair = round(float(split[-1]) , 5 )
#     if ((fina[0] < fina[1]) and (init[0] > init[1])):
#         obs_twobody_df.loc[counter] = [init[0],init[1],fina[0],fina[1],V_pair]
#     else:
#         continue

# dum_len = len(obs_twobody_df)

# for counter, i in enumerate(text_two_prot):
#     split = i.split()
#     fina = (int(split[0])-1+num_neut_orb,int(split[2])-1+num_neut_orb);
#     init =  (int(split[4])-1+num_neut_orb,int(split[6])-1+num_neut_orb) #  -1 because start from 0, plus num_neut_orb because proton comes after neutron's
#     V_pair = round(float(split[-1]) , 5 ) 
#     if ((fina[0] < fina[1]) and (init[0] > init[1])):
#         obs_twobody_df.loc[counter+dum_len] = [init[0],init[1],fina[0],fina[1],V_pair]
#     else:
#         continue

# # to process all possible values (60a1)
# obs_twobody_df = pd.DataFrame(columns =["q_i1", "q_i2", "q_f1", "q_f2", "V_ffii"])

# # taking the first two as init and the last 2 as final.
# count = 0
# for counter, i in enumerate(text_two_neut):
#     split = i.split()
#     fina = (int(split[0])-1,int(split[2])-1); 
#     init =  (int(split[4])-1,int(split[6])-1)
#     if (abs(float(split[-1])) < 0.0001) or (fina not in pairlist) or (init not in pairlist):
#         count +=1 
#         pass
#     else:
#         V_pair = round(float(split[-1]) , 5 )
#         obs_twobody_df.loc[counter] = [init[0],init[1],fina[0],fina[1],V_pair]

# dum_len = len(obs_twobody_df)

# for counter, i in enumerate(text_two_prot):
#     split = i.split()
#     fina = (int(split[0])-1+num_neut_orb,int(split[2])-1+num_neut_orb);
#     init =  (int(split[4])-1+num_neut_orb,int(split[6])-1+num_neut_orb) #  -1 because start from 0, plus num_neut_orb because proton comes after neutron's
#     if (abs(float(split[-1])) <0.0001) or (fina not in pairlist) or (init not in pairlist):
#         count +=1 
#         pass
#     else:
#         V_pair = round(float(split[-1]) , 5 )
#         obs_twobody_df.loc[counter+dum_len] = [init[0],init[1],fina[0],fina[1],V_pair]
